In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
import warnings
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import yfinance as yf
import datetime as dt

In [2]:
start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = yf.download('AXISBANK.NS', start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [4]:
#column="Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2144, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1715
X_train length: (1715, 4)
X_test length: (429, 4)
y_train length: (1715, 4)
y_test length: (429, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [11]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [12]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -38.38738266264952
BIC:  -38.33389036864025

Order = 2
AIC:  -38.450915630190586
BIC:  -38.35459200767806

Order = 3
AIC:  -38.53363996963472
BIC:  -38.39445164549083

Order = 4
AIC:  -38.54430148817139
BIC:  -38.3622150469554

Order = 5
AIC:  -38.55457651591017
BIC:  -38.329558499794935

Order = 6
AIC:  -38.59126575719953
BIC:  -38.32328266589763

Order = 7
AIC:  -38.59216290315327
BIC:  -38.28118119384292

Order = 8
AIC:  -38.58938151764516
BIC:  -38.23536760489597

Order = 9
AIC:  -38.59309228655474
BIC:  -38.19601254225336

Order = 10
AIC:  -38.58795066403028
BIC:  -38.14777141730578



In [13]:
data1

,Open,High,Low,Close
0,0.185361,0.176546,0.192220,0.188227
1,0.187095,0.178252,0.185383,0.177944
2,0.175532,0.165572,0.177981,0.174110
3,0.172352,0.173646,0.179809,0.184524
4,0.182470,0.171983,0.180062,0.172948
...,...,...,...,...
2139,0.925633,0.911301,0.949017,0.932392
2140,0.933655,0.910021,0.938538,0.919393
2141,0.921369,0.906325,0.926721,0.932392
2142,0.931487,0.918408,0.962394,0.940090


In [14]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.117318,9,28
